In [3]:
import os
import shutil
import numpy as np
import pandas as pd
import calendar
from datetime import datetime
from fpdf import FPDF

import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams["axes.spines.top"] = False
rcParams["axes.spines.right"] = False

In [4]:
def generate_sales_data(month: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=datetime(year=2020, month=month, day=1),
        end=datetime(year=2020, month=month,day=calendar.monthrange(2020,month)[1])
    )

    sales = np.random.randint(low=1000, high=2000, size=len(dates))

    return pd.DataFrame({
        'Date': dates,
        "ItemSold": sales
    })

In [34]:
generate_sales_data(month=5)

,Date,ItemSold
0,2020-05-01,1053
1,2020-05-02,1307
2,2020-05-03,1911
3,2020-05-04,1797
4,2020-05-05,1861
5,2020-05-06,1060
6,2020-05-07,1630
7,2020-05-08,1364
8,2020-05-09,1164
9,2020-05-10,1577


In [25]:
def plot(data: pd.DataFrame, filename: str) -> None:
    plt.figure(figsize=(12, 4))
    plt.grid(color='#F2F2F2', alpha=1, zorder=0)
    plt.plot(data['Date'], data['ItemSold'], color='#087E8B', lw=3, zorder=5)
    plt.title(f'Sales 2020/{data["Date"].dt.month[0]}', fontsize=17)
    plt.xlabel('Peroid', fontsize=13)
    plt.xticks(fontsize=9)
    plt.ylabel('Number of items sold', fontsize=13)
    plt.yticks(fontsize=9)
    plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()
    return

In [ ]:
december = generate_sales_data(month=12)
plot(data=december, filename='december.png')

In [31]:
PLOT_DIR = "plots"

def construct():
    try:
        shutil.rmtree(PLOT_DIR)
        os.mkdir(PLOT_DIR)
    except:
        os.mkdir(PLOT_DIR)

    for i in range(2, 13):
        plot(data=generate_sales_data(month=i), filename=f'{PLOT_DIR}/{i}.png')

    counter = 0
    pages_data = []
    temp = []
    files = os.listdir(PLOT_DIR)
    files = sorted(os.listdir(PLOT_DIR), key=lambda x : int(x.split('.')[0]))

    for fname in files:
        if counter == 3:
            pages_data.append(temp)
            temp = []
            counter = 0
        temp.append(f'{PLOT_DIR}/{fname}')
        counter += 1

    return [*pages_data, temp]

In [19]:
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.WIDTH = 210
        self.HEIGHT = 297

    def header(self):
        # self.image('assets/logo.png', 10, 8, 33)
        self.set_font('Arial', 'B', 11)
        self.cell(self.WIDTH - 80)
        self.cell(60, 1, 'Sales report', 0, 0, 'R')
        self.ln(20)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def page_body(self, images):
        if len(images) == 3:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
            self.image(images[2], 15, self.WIDTH / 2 + 90, self.WIDTH - 30)
        elif len(images) == 2:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
        else:
            self.image(images[0], 15, 25, self.WIDTH - 30)

    def print_page(self, images):
        self.add_page()
        self.page_body(images)

In [33]:
plots_per_page = construct()
print(plots_per_page)

[['plots/2.png', 'plots/3.png', 'plots/4.png'], ['plots/5.png', 'plots/6.png', 'plots/7.png'], ['plots/8.png', 'plots/9.png', 'plots/10.png'], ['plots/11.png', 'plots/12.png']]


In [20]:
pdf = PDF()

for elem in plots_per_page:
    pdf.print_page(elem)

pdf.output('sales_report.pdf', 'F')

''